In [77]:
from glom import glom, Coalesce
from elasticsearch import Elasticsearch, helpers
import eland as ed
import pandas as pd
from itertools import chain
from collections import Counter

import pyecharts

In [4]:
# es = Elasticsearch(
#     "https://elasticsearch:9200",
#     api_key="YjVwMHpKSUI1aVdrRE5nRHhQN0o6UkxLTGRoUGRSVDZic3NCU2IzNFVnQQ==",
#     verify_certs=False,
# )
es = Elasticsearch(
    "http://host.docker.internal:9200",
    verify_certs=False,
    basic_auth=("elastic", "123456"),
)

In [121]:
q_word = "A酸"
body = {
    # "_source": False,
    "_source": ["title_aste.*", "context_aste.*", "date", "comments.content_aste.*"],
    # "fields": ["title_aste.*", "context_aste.*", "date", "comments.content_aste.*"],
    "query": {
        "multi_match": {
            "query": q_word,
            "fields": ["title", "context"],
            "minimum_should_match": "50%",
        },
    },
}

In [122]:
tmp = es.search(index="dcard", body=body)

In [123]:
tmp.body

{'took': 23,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 57, 'relation': 'eq'},
  'max_score': 7.9166183,
  'hits': [{'_index': 'dcard',
    '_id': '257101254',
    '_score': 7.9166183,
    '_source': {'date': '2024-10-30 21:41:55.653',
     'context_aste': [{'p': 4.7196, 'a': '副作用', 'o': '大'},
      {'p': 3.8299999999999996, 'a': '禮拜', 'o': '擦'}]}},
   {'_index': 'dcard',
    '_id': '257100380',
    '_score': 7.536421,
    '_source': {'date': '2024-10-30 20:15:54.174',
     'context_aste': [{'p': 4.7196, 'a': '負擔', 'o': '大'}],
     'title_aste': [{'p': 2.9212, 'a': '北部', 'o': '服'}]}},
   {'_index': 'dcard',
    '_id': '257078826',
    '_score': 6.8930225,
    '_source': {'date': '2024-10-28 14:11:50.262',
     'comments': [{'content_aste': [{'p': 4.9894, 'a': '懶惰', 'o': '停'},
        {'p': 6.3, 'a': '效果', 'o': '不錯'}]},
      {'content_aste': [{'p': 6.6, 'a': '膚況', 'o': '進步'},
        {'p': 4.6334, 'a': '我', 'o

In [126]:
coalesce_of_path = Coalesce(
    "hits.hits.*._source.context_aste",
    "hits.hits.*._source.title_aste",
    "hits.hits.*._source.comments.*.content_aste",
)
# glom(tmp.body, coalesce_of_path)
# glom(tmp.body, "hits.hits.*.fields.context_aste.*")
df = pd.DataFrame.from_dict(list(chain.from_iterable(glom(tmp.body, coalesce_of_path))))

In [130]:
c_item = Counter(df["a"].to_list()).items()

In [131]:
c_item

dict_items([('副作用', 1), ('禮拜', 1), ('負擔', 1), ('痘痘肌', 2), ('人', 9), ('毛孔', 4), ('強度', 1), ('新手', 4), ('你', 4), ('注意', 1), ('水楊酸', 1), ('果酸', 1), ('膚色', 2), ('頻率', 2), ('肌膚', 1), ('杏仁酸', 2), ('我', 21), ('官網', 1), ('說', 1), ('說明', 1), ('精華', 2), ('蘋果酸', 1), ('甘醇酸', 1), ('分子', 1), ('效果', 5), ('三重酸', 1), ('特點', 1), ('個人', 2), ('刺激性', 1), ('文章', 1), ('功能', 1), ('敏感肌', 1), ('媽媽', 1), ('疫情', 1), ('臉', 6), ('時候', 6), ('痘痘', 2), ('手', 1), ('圖片', 1), ('吸管', 1), ('討厭', 1), ('下午', 1), ('肚子', 1), ('後面', 1), ('自己', 4), ('生理期', 1), ('大家', 1), ('我們', 1), ('題', 2), ('汗', 1), ('味道', 2), ('電腦', 1), ('室友', 1), ('方法', 1), ('爬文', 2), ('皮膚', 1), ('他', 2), ('醫生', 2), ('底下', 1), ('紅點', 1), ('雀斑', 1), ('原因', 1), ('黑眼圈', 1), ('眼睛', 1), ('邊', 1), ('同事', 1), ('壓力', 1), ('水', 1), ('月', 1), ('藥局', 1), ('凝膠', 1), ('杜鵑花酸', 1), ('天氣', 4), ('火山泥', 1), ('潔面乳', 1), ('裡面', 1), ('泡沫', 1), ('白茶', 1), ('雪臻', 1)])

In [168]:
from pyecharts.charts import WordCloud
from pyecharts import options as opts
from streamlit_echarts import st_pyecharts
import streamlit as st

In [171]:
wc = (
    WordCloud(opts.InitOpts(chart_id="ws"))
    .add(series_name="A酸", data_pair=c_item)
    .add_js_funcs("""
    chart_ws.on('click', function (params) { console.log(params); })
    """)
)

In [175]:
import json

In [178]:
with open("wc.json", "w") as f:
    f.write(wc.dump_options())


In [176]:
json.decoder.JSONDecoder.decode(wc.dump_options())

TypeError: JSONDecoder.decode() missing 1 required positional argument: 's'

In [173]:
wc.dump_options_with_quotes()

'{\n    "animation": true,\n    "animationThreshold": 2000,\n    "animationDuration": 1000,\n    "animationEasing": "cubicOut",\n    "animationDelay": 0,\n    "animationDurationUpdate": 300,\n    "animationEasingUpdate": "cubicOut",\n    "animationDelayUpdate": 0,\n    "aria": {\n        "enabled": false\n    },\n    "color": [\n        "#5470c6",\n        "#91cc75",\n        "#fac858",\n        "#ee6666",\n        "#73c0de",\n        "#3ba272",\n        "#fc8452",\n        "#9a60b4",\n        "#ea7ccc"\n    ],\n    "series": [\n        {\n            "type": "wordCloud",\n            "name": "A\\u9178",\n            "shape": "circle",\n            "rotationRange": [\n                -90,\n                90\n            ],\n            "rotationStep": 45,\n            "girdSize": 20,\n            "sizeRange": [\n                12,\n                60\n            ],\n            "data": [\n                {\n                    "name": "\\u526f\\u4f5c\\u7528",\n                    "v

In [172]:
wc.render_notebook()

In [104]:
tmp = es.search(
    index="dcard",
    body=body,
    filter_path=["hits.hits.inner_hits.*.hits.hits._source.*", "hits.total.value"],
)
tmp = list(chain.from_iterable(glom(tmp.body, "hits.hits.**.hits.hits.*._source")))

/usr/local/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elasticsearch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [142]:
tag_counter = Counter(tmp["t"])

pos_o_counter = Counter(tmp.o[tmp.t == "POS"])
neg_o_counter = Counter(tmp.o[tmp.t == "NEG"])
nat_o_counter = Counter(tmp.o[tmp.t == "NAT"])

In [143]:
tag_counter.items()

dict_items([('NAT', 4), ('POS', 1)])

In [144]:
def aop_df_2_data(aop_df):
    tag_counter = Counter(aop_df["t"])
    return [
        {
            "value": v,
            "name": k,
            "children": [
                {"value": c, "name": w}
                for w, c in Counter(aop_df.o[aop_df.t == k]).items()
            ],
        }
        for k, v in tag_counter.items()
    ]

In [145]:
aop_df_2_data(tmp)

[{'value': 4,
  'name': 'NAT',
  'children': [{'value': 1, 'name': '口服'},
   {'value': 1, 'name': '減少'},
   {'value': 1, 'name': '去'},
   {'value': 1, 'name': '吃'}]},
 {'value': 1, 'name': 'POS', 'children': [{'value': 1, 'name': '好用'}]}]